In [213]:
import os
import numpy as np
import json
import cv2
import shutil
import pandas as pd
import time

from pathlib import Path

# 동영상 -> 이미지 변환 코드

In [195]:
def generate_paths(datapath):
    if not os.path.exists(convert_filepath):
        os.mkdir(convert_filepath)
    
    if not os.path.exists(os.path.join(convert_filepath, 'images')):
        os.mkdir(os.path.join(convert_filepath, 'images'))
        
    if not os.path.exists(os.path.join(convert_filepath, 'labels_with_ids')):
        os.mkdir(os.path.join(convert_filepath, 'labels_with_ids'))

In [196]:
def convert_video2img(video_filename, convert_filepath):
    video = cv2.VideoCapture(video_filename)
    
    length = video.get(cv2.CAP_PROP_POS_MSEC)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)
    
    frame_idx = 0
    while(video.isOpened()):
        
        ret, frame = video.read()
        if not ret:
            break
        
        frame_idx += 1
        filename = 'image_%04d.png' % (frame_idx)
        print('\tfilename: %s' % (filename), end='\r')
        
        img_path = os.path.join(convert_filepath, 'images', filename).replace('\\', '/') 
        cv2.imwrite(img_path, frame)
        
    print()

In [197]:
VIDEO_PATH = 'C:/Users/rohyujin/Desktop/서류' # 동영상이 저장되어 있는 경로
VIDEO_FILE_LIST = [os.path.join(VIDEO_PATH, _).replace('\\', '/') for _ in sorted(os.listdir(VIDEO_PATH))]
CONVERT_PATH = '../datasets/kw_video' # 동영상을 변환해서 저장해둘 경로


if not os.path.exists(CONVERT_PATH):
    os.mkdir(CONVERT_PATH)

In [201]:
print('='*40, 'Video to Image', '='*40)
start_time = time.time()
for video_index, video_filename in enumerate(VIDEO_FILE_LIST):
    video_index = video_index + 1
    
    video_name = 'video_%02d' % video_index
    convert_filepath = os.path.join(CONVERT_PATH, video_name).replace('\\', '/')
    
    print('\tTarget path: %s\n\tConvert path: %s' % (video_filename, convert_filepath))
    
    generate_paths(convert_filepath)
    convert_start_time = time.time()
    convert_video2img(video_filename, convert_filepath)
    convert_end_time = time.time()
    print("\t%s file conversion took %.2f mins." % (video_name, (convert_end_time - convert_start_time)/60))
    print('')
    
end_time = time.time()
print('='*40, 'All video file has been converted', '='*40)
print('Total converting time so far: %.2f mins' % ((end_time-start_time)/60))

======================================== Video to Image ========================================
	Target path: C:/Users/rohyujin/Desktop/서류/KakaoTalk_20221111_125640346.mp4
	Convert path: ../datasets/kw_video/video_01
	filename: image_0298.png
	video_01 file conversion took 0.19 mins.

	Target path: C:/Users/rohyujin/Desktop/서류/KakaoTalk_20221111_125649287.mp4
	Convert path: ../datasets/kw_video/video_02
	filename: image_0335.png
	video_02 file conversion took 0.20 mins.

	Target path: C:/Users/rohyujin/Desktop/서류/KakaoTalk_20221111_125653827.mp4
	Convert path: ../datasets/kw_video/video_03
	filename: image_0368.png
	video_03 file conversion took 0.22 mins.

	Target path: C:/Users/rohyujin/Desktop/서류/KakaoTalk_20221111_125703126.mp4
	Convert path: ../datasets/kw_video/video_04
	filename: image_0265.png
	video_04 file conversion took 0.16 mins.

	Target path: C:/Users/rohyujin/Desktop/서류/KakaoTalk_20221111_125706501.mp4
	Convert path: ../datasets/kw_video/video_05
	filename: image_0276.

# 변환된 이미지를 이용하여 오토 어노테이션 해보기<br>

python tools/demo_track_custom.py image --path datasets/kw_video/video_01 -f exps/example/mot/yolox_x_ablation.py -c pretrained/bytetrack_ablation.pth.tar --fp16 --fuse --save_result

In [536]:
YOLOX_RESULT_PATH = '../YOLOX_outputs/yolox_x_ablation/track_vis/'
TARGET_TXT_LIST = [os.path.join(YOLOX_RESULT_PATH, _).replace('\\', '/') for _ in sorted(os.listdir(YOLOX_RESULT_PATH)) if 'video' in _ and 'txt' in _]
AUTO_ANNOTATION_PATH = os.path.join(CONVERT_PATH, 'auto_anno').replace('\\', '/')
IMG_PATH = os.path.join(CONVERT_PATH, '{video_filepath}/images').replace('\\', '/')

Path(AUTO_ANNOTATION_PATH).mkdir(parents=True, exist_ok=True)

# image_id = 1
video_id = 1

cross_walk_bbox_list = [
    [0, 470, 370, 190],
    [0, 450, 510, 160]
]

for txt_filepath in TARGET_TXT_LIST:
    print(txt_filepath)
    out = {'images': [], 'annotations': [], 'videos': [], 'categories': [{'id': 2, 'name': 'crosswalk'}, {'id': 1, 'name': 'pedestrian'}]}
    video_filepath = os.path.basename(txt_filepath)[:-4]
    image_path = IMG_PATH.format(video_filepath = video_filepath)
    dir_name = image_path.split(CONVERT_PATH)[-1][1:]
    
    image_file_list = [os.path.join(image_path, _).replace('\\', '/') for _ in sorted(os.listdir(image_path)) if '.png' in _]
    image_id = 0
    
    cross_walk_bbox = cross_walk_bbox_list[video_id-1]
    
    for frame_id, image_filename in enumerate(image_file_list):
        
        _filename = os.path.basename(image_filename) #os.path.join(dir_name, os.path.basename(image_filename)).replace('\\', '/')
        _id = image_id
        _frame_id = frame_id #+1
        _prev_image_id = _frame_id -1 if _frame_id > 0 else -1
        _next_image_id = _frame_id +1 if _frame_id <= len(image_file_list) else -1
        _video_id = video_id
        
        img = cv2.imread(image_filename, cv2.IMREAD_GRAYSCALE)
        
        _height = img.shape[0]
        _width = img.shape[1]
        
        image_info = {
            'file_name': _filename, 
            'id': _id,
            'frame_id': _frame_id, 
            'prev_image_id': _prev_image_id, 
            'next_image_id': _next_image_id, 
            'video_id': _video_id,
            'height': _height, 
            'width': _width
        }
        
        out['images'].append(image_info)
        
        ann = {
            'id': 0,
            'category_id': 1,
            'image_id': _id,
            'track_id': 0,
            'bbox': cross_walk_bbox,
            'conf': 1.0,
            'segmentation': [],
            'iscrowd': 0,
            'area': cross_walk_bbox[2] * cross_walk_bbox[3],
        }
        
        out['annotations'].append(ann)
        
        image_id += 1
    
    with open(txt_filepath, 'r') as f:
        strings = f.readlines()
    
    for string in strings:
        string = string.replace('\n', '')
        _image_id, _id, x, y, w, h, _, _, _, _ = string.split(',')
        
        _id = int(_id)
        _image_id = int(_image_id)-1
        _bbox = [float(x), float(y), float(w), float(h)]
        _conf = 1.0
        _iscrowd = 0
        _area = float(w) * float(h)
        
        if video_id == 2 and (_id == 9 or _id == 11 or _id == 17 or _id == 18):
            continue
        
        if video_id == 2 and (_id == 8 or _id == 10):
            _id = 8
            if _image_id == 9:
                for i in range(_image_id):
                    ann = {
                        'id': _id,
                        'category_id': 2,
                        'image_id': i,
                        'track_id': _id,
                        'bbox': _bbox,
                        'conf': _conf,
                        'segmentation': [],
                        'iscrowd': _iscrowd,
                        'area': _area,
                    }
                    
                    out['annotations'].append(ann)
            if _image_id == 57:
                for i in range(15, _image_id):
                    ann = {
                        'id': _id,
                        'category_id': 2,
                        'image_id': i,
                        'track_id': _id,
                        'bbox': _bbox,
                        'conf': _conf,
                        'segmentation': [],
                        'iscrowd': _iscrowd,
                        'area': _area,
                    }
                    
                    out['annotations'].append(ann)
        if video_id == 2 and (_id == 7):
            if _image_id == 105:
                for i in range(99, _image_id):
                    ann = {
                        'id': _id,
                        'category_id': 2,
                        'image_id': i,
                        'track_id': _id,
                        'bbox': _bbox,
                        'conf': _conf,
                        'segmentation': [],
                        'iscrowd': _iscrowd,
                        'area': _area,
                    }
                    
                    out['annotations'].append(ann)
            
        if video_id == 2 and (_id == 12 or _id == 19):
            _id = 3
            
        if video_id == 2 and _id == 13:
            _id = 9
            
        if video_id == 2 and _id == 14:
            _id = 10
        
        _track_id = _id
        
        ann = {
            'id': _id,
            'category_id': 2,
            'image_id': _image_id,
            'track_id': _track_id,
            'bbox': _bbox,
            'conf': _conf,
            'segmentation': [],
            'iscrowd': _iscrowd,
            'area': _area,
        }
        
        out['annotations'].append(ann)
    
    
    s = {}
    for i, _ in enumerate(out['annotations']):
        s[i] = _['image_id']

    sorted_dict = dict(sorted(s.items(), key = lambda item: item[1])) #dict(sorted({value: key for key, value in s.items()}.items()))
    sorted_out_anno = []
    for key, value in sorted_dict.items():
        sorted_out_anno.append(out['annotations'][key])
        
    out['annotations'] = sorted_out_anno
    
    video_info = {
        'id': 1,
        'file_name': video_filepath
    }
    
    out['videos'].append(video_info)
    
    json_filepath = os.path.join(AUTO_ANNOTATION_PATH, video_filepath + '.json')
    json.dump(out, open(json_filepath, 'w'), indent=4)
    
    video_id += 1

../YOLOX_outputs/yolox_x_ablation/track_vis/video_01.txt
../YOLOX_outputs/yolox_x_ablation/track_vis/video_04.txt


# ※ Roboflow 이용하여 이미지 어노테이션 및 성능 평가를 위한 파일 변환 작업
## https://app.roboflow.com/kwanwoon-univerty 에 접속하여, 변환하고자 하는 이미지를 업로드 후 어노테이션<br>
### 1. 카테고리
0은 human, 1은 횡단보도(crosswalk), 2는 보행자(pedestrian)으로 설정 <br>
ex) {'id': 0, 'name': 'human', 'supercategory': 'none'}, <br>
    {'id': 1, 'name': 'crosswalk', 'supercategory': 'human'}, <br>
    {'id': 2, 'name': 'pedestrian', 'supercategory': 'human'} <br>

### 2. 어노테이션이 완료된 json 파일은 이미지가 저장되어 있는 곳의 상위 폴더의 origin_json 폴더에 저장
ex) 이미지 저장 폴더 => C:/Users/rohyujin/workspace/bytetrack/datasets/kw_video/video_01/images <br>
    어노테이션 저장 폴더 => C:/Users/rohyujin/workspace/bytetrack/datasets/kw_video/origin_json <br>
    origin_json 폴더가 없을 시 직접 생성 <br>
    
### 3. 폴더 구조


In [459]:
CONVERT_PATH = CONVERT_PATH # '../datasets/kw_video' # 변환 필요
ORIGIN_JSON_PATH = os.path.join(CONVERT_PATH, 'origin_anno').replace('\\', '/')

json_file_list = [_ for _ in sorted(os.listdir(ORIGIN_JSON_PATH)) if '.json' in _ and 'sorted' not in _]

for json_filename in json_file_list:
    out = {'images': [], 'annotations': [], 'categories': []}
    
    video_id = json_filename[:-5]
    json_filepath = os.path.join(ORIGIN_JSON_PATH, json_filename).replace('\\', '/')
    Path(TXT_PATH.format(video_id = video_id)).mkdir(parents=True, exist_ok=True)
    with open(json_filepath, 'r') as json_file:
        json_data = json.load(json_file)
    out['categories'] = json_data['categories']
    
    s = {}
    for _image_info in json_data['images']:
        _filename = _image_info['file_name'].split('.rf')[0][:-4] + '.png'
        _id = int(_filename.split('_')[1].split('.png')[0]) -1
        _height = _image_info['height']
        _width = _image_info['width']

        image_info = {
            'id': _id,
            'file_name': _filename, 
            'height': _height, 
            'width': _width
        }
        
        out['images'].append(image_info)
        
        s[_filename] = _image_info['id']

    id_sort_dict = dict(sorted(s.items()))
    file_sort_dict = {value: int(key.split('_')[1].split('.png')[0]) -1 for key, value in id_sort_dict.items()}
    
    for _anno in json_data['annotations']:
        _id = _anno['id']
        _category_id = _anno['category_id']
        _image_id = file_sort_dict[_anno['image_id']] #_anno['image_id'] + 1
        _bbox = [int(_) for _ in _anno['bbox']]
        _iscrowd = _anno['iscrowd']
        _area = _bbox[2] * _bbox[3]
        x,y,w,h = _bbox

        ann = {
            'id': _id,
            'category_id': _category_id,
            'image_id': _image_id,
            'bbox': _bbox,
            'iscrowd': _iscrowd,
            'area': _area,
        }

        out['annotations'].append(ann)
    
    s = {}
    for i, _ in enumerate(out['images']):
        s[i] = _['id']

    sorted_dict = dict(sorted({value: key for key, value in s.items()}.items()))
    sorted_out_images = []
    for key, value in sorted_dict.items():
        sorted_out_images.append(out['images'][value])

    out['images'] = sorted_out_images
    
    s = {}
    for i, _ in enumerate(out['annotations']):
        s[i] = _['image_id']

    sorted_dict = dict(sorted(s.items(), key = lambda item: item[1])) #dict(sorted({value: key for key, value in s.items()}.items()))
    sorted_out_anno = []
    for key, value in sorted_dict.items():
        sorted_out_anno.append(out['annotations'][key])
        
    out['annotations'] = sorted_out_anno
    
    _id_idx = 1
    out_anno_list = []
    for _anno in out['annotations']:
        _category_id = _anno['category_id']

        if _category_id == 1:
            _anno['id'] = 0

        elif _category_id == 2:
            _anno['id'] = _id_idx
            _id_idx += 1

        out_anno_list.append(_anno)

    out['annotations'] = out_anno_list
    
    
    out_path = os.path.join(ORIGIN_JSON_PATH, 'sorted_' + video_id + '.json').replace('\\', '/')

    json.dump(out, open(out_path, 'w'), indent=4)

In [538]:
CONVERT_PATH = CONVERT_PATH # '../datasets/kw_video' # 변환 필요
ORIGIN_JSON_PATH = os.path.join(CONVERT_PATH, 'origin_anno').replace('\\', '/')

OUT_PATH = os.path.join(CONVERT_PATH, 'annotations/').replace('\\', '/')
IMG_PATH = os.path.join(CONVERT_PATH, '{video_id}/images').replace('\\', '/')
TXT_PATH = os.path.join(CONVERT_PATH, '{video_id}/labels_with_ids').replace('\\', '/') # 변환 필요

Path(OUT_PATH).mkdir(parents=True, exist_ok=True)

json_file_list = [_ for _ in sorted(os.listdir(ORIGIN_JSON_PATH)) if '.json' in _ and 'sorted' in _]

video = 0
for idx, json_filename in enumerate(json_file_list):
    video_id = json_filename[:-5].split('sorted_')[1]
    json_filepath = os.path.join(ORIGIN_JSON_PATH, json_filename).replace('\\', '/')
    Path(TXT_PATH.format(video_id = video_id)).mkdir(parents=True, exist_ok=True)
    _txt_filename = video_id + '.txt'
    _txt_filepath = os.path.join(TXT_PATH.format(video_id=video_id), _txt_filename)
    
    image_id = 1
    image_len = 0
    out = {'images': [], 'annotations': [], 'videos': [], 'categories': [{'id': 1, 'name': 'pedestrian'}]}
    
    if os.path.exists(_txt_filepath):
        os.remove(_txt_filepath)
    
    with open(os.path.join(ORIGIN_JSON_PATH, json_filename), 'r') as json_file:
        json_data = json.load(json_file)
        
    for _image_info in json_data['images']:
        _filename = os.path.join(video_id, os.path.basename(IMG_PATH.format(video_id=video_id)), _image_info['file_name']).replace('\\', '/')
        _id = image_id #_image_info['id'] + 1 # 나중에 변경 필요, 비디오가 여러개일 경우에 변경해야 함
        _frame_id = _image_info['id'] + 1
        _prev_image_id = _frame_id -1 if _frame_id > 1 else -1
        _next_image_id = _frame_id +1 if _frame_id < len(json_data['images']) else -1
        _video_id = 1 #idx+1 # 나중에 변경 필요, 비디오가 여러개일 경우에 변경해야 함
        _height = _image_info['height']
        _width = _image_info['width']

        image_info = {
            'file_name': _filename, 
            'id': _id,
            'frame_id': _frame_id, #frame_idx,
            'prev_image_id': _prev_image_id, # image_cnt-1 if frame_idx > 1 else -1,
            'next_image_id': _next_image_id, #image_cnt+1 if frame_idx <= len(json_data['images'])+1 else -1,
            'video_id': _video_id,
            'height': _height, 
            'width': _width
        }

        out['images'].append(image_info)
        
        image_id += 1

    for _anno in json_data['annotations']:
        _category_id = _anno['category_id']
        if _category_id == 2:
            _id = _anno['id'] #+ 1
            _category_id = 1
            _image_id = _anno['image_id'] + 1 + image_len
            _track_id = _anno['id']
            _bbox = [int(_) for _ in _anno['bbox']]
            _conf = 1.0
            _iscrowd = _anno['iscrowd']
            _area = _bbox[2] * _bbox[3]
            x,y,w,h = _bbox

            ann = {
                'id': _id,
                'category_id': _category_id,
                'image_id': _image_id,
                'track_id': _track_id,
                'bbox': _bbox,
                'conf': _conf,
                'iscrowd': _iscrowd,
                'area': _area,
            }

            out['annotations'].append(ann)
            
            # _txt_filename = json_data['images'][_anno['image_id']]['file_name'].split('.png')[0] + '.txt'

            with open(_txt_filepath, 'a') as f:
                txt = f'{_image_id},{_id},{x},{y},{w},{h},1,-1,-1,-1\n'
                f.write(txt)

    video_info = {
        'id': 1, #idx+1,
        'file_name': video_id #os.path.basename(CONVERT_PATH)
    }        
    out['videos'].append(video_info)

    image_len = image_id -1
    
    out_path = OUT_PATH + 'val_' + video_id + '.json'

    json.dump(out, open(out_path, 'w'))

python tools/track_custom.py -f exps/example/mot/yolox_x_custom.py -c YOLOX_outputs/yolox_x_ablation/best_ckpt.pth.tar -b 1 -d 1 --fp16 --fuse

In [140]:
# json_filename = os.path.basename(os.path.dirname(IMG_PATH)) + '_annotations.json' #os.listdir(ORIGIN_JSON_PATH)[0]
# print('json_filename is', json_filename)

json_filename is video_01_annotations.json


In [141]:
# with open(os.path.join(ORIGIN_JSON_PATH, json_filename), 'r') as json_file:
#     json_data = json.load(json_file)

In [142]:
# out = {'images': [], 'annotations': [], 'videos': [], 'categories': [{'id': 1, 'name': 'pedestrian'}]}

# for _image_info in json_data['images']:
#     _filename = os.path.join(os.path.basename(IMG_PATH), _image_info['file_name']).replace('\\', '/')
#     _id = _image_info['id'] + 1 # 나중에 변경 필요, 비디오가 여러개일 경우에 변경해야 함
#     _frame_id = _image_info['id'] + 1
#     _prev_image_id = _frame_id -1 if _frame_id > 1 else -1
#     _next_image_id = _frame_id +1 if _frame_id < len(json_data['images']) else -1
#     _video_id = 1 # 나중에 변경 필요, 비디오가 여러개일 경우에 변경해야 함
#     _height = _image_info['height']
#     _width = _image_info['width']
    
#     image_info = {
#         'file_name': _filename, 
#         'id': _id,
#         'frame_id': _frame_id, #frame_idx,
#         'prev_image_id': _prev_image_id, # image_cnt-1 if frame_idx > 1 else -1,
#         'next_image_id': _next_image_id, #image_cnt+1 if frame_idx <= len(json_data['images'])+1 else -1,
#         'video_id': _video_id,
#         'height': _height, 
#         'width': _width
#     }
    
#     out['images'].append(image_info)
    
# for _anno in json_data['annotations']:
#     _category_id = _anno['category_id']
#     if _category_id == 2:
#         _id = _anno['id'] + 1
#         _category_id = 1
#         _image_id = _anno['image_id'] + 1
#         _track_id = -1
#         _bbox = [int(_) for _ in _anno['bbox']]
#         _conf = 1.0
#         _iscrowd = _anno['iscrowd']
#         _area = _bbox[2] * _bbox[3]
#         x,y,w,h = _bbox
    
#         ann = {
#             'id': _id,
#             'category_id': _category_id,
#             'image_id': _image_id,
#             'track_id': _track_id,
#             'bbox': _bbox,
#             'conf': _conf,
#             'iscrowd': _iscrowd,
#             'area': _area,
#         }
        
#         out['annotations'].append(ann)
        
#         _txt_filename = json_data['images'][_anno['image_id']]['file_name'].split('.jpg')[0] + '.txt'

#         with open(os.path.join(TXT_PATH, _txt_filename), 'a') as f:
#             txt = f'{_id},{_image_id},{x},{y},{w},{h},-1,-1,-1,-1\n'
#             f.write(txt)

# video_info = {
#     'id': 1,
#     'file_name': os.path.basename(CONVERT_PATH)
# }        
# out['videos'].append(video_info)

# out_path = OUT_PATH + 'val.json'

# json.dump(out, open(out_path, 'w'))

In [58]:
# if __name__ == '__main__':
#     AVI_FILE_LIST = [os.path.join(DATA_PATH, _) for _ in os.listdir(DATA_PATH) if '.avi' in _]
#     image_cnt = 0
#     out_path = OUT_PATH + 'train.json'
#     out = {'images': [], 'annotations': [], 'videos': [], 'categories': [{'id': 1, 'name': 'pedestrian'}]}

#     for avi_idx, avi_filenbame in enumerate(AVI_FILE_LIST):
#         generate_paths(avi_filenbame)

#         video = cv2.VideoCapture(avi_filenbame)

#         file_basename = os.path.basename(avi_filenbame).split('.avi')[0]
#         csv_filepath = avi_filenbame.replace('.avi', '.csv')
#         new_csv_filepath = os.path.join(CONVERT_PATH, file_basename, 'labels_with_ids', file_basename+'.csv')
#         shutil.copy(csv_filepath, new_csv_filepath)

#         frame_idx = 1
        
#         df = pd.read_csv(new_csv_filepath)
#         length = video.get(cv2.CAP_PROP_POS_MSEC)
#         width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
#         height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
#         fps = video.get(cv2.CAP_PROP_FPS)
        
#         while(video.isOpened()):
#             ret, frame = video.read()

#             if not ret:
#                 break
#             print('filename: %s\tframe_number: %04d' % (file_basename, frame_idx), end='\r')
            
#             image_cnt += 1
#             img_path = os.path.join(CONVERT_PATH, file_basename, 'images', 'image_%04d.png') % (frame_idx)
#             img_path = img_path.replace('\\', '/')
# #             cv2.imwrite(img_path, frame)

#             image_info = {
#                 'file_name': ''.join(img_path.split(CONVERT_PATH)[1:]), 
#                 'id': image_cnt,
#                 'frame_id': frame_idx,
#                 'prev_image_id': image_cnt-1 if frame_idx > 1 else -1,
#                 'next_image_id': image_cnt+1 if frame_idx <= len(df) else -1,
#                 'video_id': avi_idx+1,
#                 'height': frame.shape[0], 
#                 'width': frame.shape[1]
#             }

#             out['images'].append(image_info)

#             labels0 = df.iloc[frame_idx-1]
#             labels0[labels0<0] = 0
#             labels = labels0.copy()
#             fbox = labels.tolist()
#             ann = {
#                 'id': image_cnt,
#                 'category_id': 1,
#                 'image_id': frame_idx,
#                 'track_id': -1,
#                 'bbox': fbox,
#                 'conf': 1.0,
#                 'iscrowd': 0,
#                 'area': fbox[2] * fbox[3],
#             }        
#             out['annotations'].append(ann)
            
#             if frame_idx == 1:
#                 video_info = {
#                     'id': avi_idx+1,
#                     'file_name': file_basename
#                 }

#                 out['videos'].append(video_info)
            
            
#             frame_idx += 1
#         print()
#         print('loaded train for {} images and {} samples'.format(len(out['images']), len(out['annotations'])))

#         json.dump(out, open(out_path, 'w'))

filename: crosswalk	frame_number: 0378
loaded train for 378 images and 378 samples
filename: fourway	frame_number: 1281
loaded train for 1659 images and 1659 samples
filename: night	frame_number: 0565
loaded train for 2224 images and 2224 samples


In [47]:
# if __name__ == '__main__':
#     AVI_FILE_LIST = [os.path.join(DATA_PATH, _) for _ in os.listdir(DATA_PATH) if '.avi' in _]
#     image_cnt = 0
#     ann_cnt = 0
#     out_path = OUT_PATH + 'train.json'
#     out = {'images': [], 'annotations': [], 'categories': [{'id': 1, 'name': 'person'}]}

#     for avi_filenbame in AVI_FILE_LIST:
#         generate_paths(avi_filenbame)

#         video = cv2.VideoCapture(avi_filenbame)

#         file_basename = os.path.basename(avi_filenbame).split('.avi')[0]
#         csv_filepath = avi_filenbame.replace('.avi', '.csv')
#         new_csv_filepath = os.path.join(CONVERT_PATH, file_basename, 'labels_with_ids', file_basename+'.csv')
#         shutil.copy(csv_filepath, new_csv_filepath)

#         frame_idx = 1

#         df = pd.read_csv(new_csv_filepath)

#         while(video.isOpened()):
#             print('filename: %s\tframe_number: %04d' % (file_basename, frame_idx), end='\r')

#             ret, frame = video.read()

#             if not ret:
#                 break

#             image_cnt += 1
#             ann_cnt += 1
#             img_path = os.path.join(CONVERT_PATH, file_basename, 'images', 'image_%04d.png') % (frame_idx)
#             img_path = img_path.replace('\\', '/')
# #             cv2.imwrite(img_path, frame)

#             image_info = {'file_name': img_path, 
#                             'id': image_cnt,
#                             'height': frame.shape[0], 
#                             'width': frame.shape[1]}

#             out['images'].append(image_info)

#             labels0 = df.iloc[frame_idx-1]

#             labels = labels0.copy()
#     #         labels[:, 2] = image_info['width'] * (labels0[:, 0]
#     #         labels[:, 3] = image_info['height'] * (labels0[:, 1]
#     #         labels[:, 4] = image_info['width'] * labels0[:, 4]
#     #         labels[:, 5] = image_info['height'] * labels0[:, 5]       
#             fbox = labels.tolist()
#             ann = {'id': ann_cnt,
#                     'category_id': 1,
#                     'image_id': image_cnt,
#                     'track_id': -1,
#                     'bbox': fbox,
#                     'area': fbox[2] * fbox[3],
#                     'iscrowd': 0}        
#             out['annotations'].append(ann)
#             frame_idx += 1
#         print()
#         print('loaded train for {} images and {} samples'.format(len(out['images']), len(out['annotations'])))
# #         json.dump(out, open(out_path, 'w'))

filename: crosswalk	frame_number: 0379
loaded train for 378 images and 378 samples
filename: fourway	frame_number: 1282
loaded train for 1659 images and 1659 samples
filename: night	frame_number: 0566
loaded train for 2224 images and 2224 samples


In [51]:
frame_idx

566